## Exercicio 1

In [1]:
import numpy as np
from scipy.stats import multivariate_normal

# Definir os parâmetros da mistura
mu1 = np.array([2, -1])
mu2 = np.array([1, 1])

sigma1 = np.array([[4, 1], [1, 4]])
sigma2 = np.array([[2, 0], [0, 2]])

pi1 = 0.5
pi2 = 0.5

# Observações
x1 = np.array([1, 0])
x2 = np.array([0, 2])
x3 = np.array([3, -1])
observations = [x1, x2, x3]

# Função para calcular as responsabilidades r_ik
# Função para calcular as responsabilidades r_ik
def calc_responsibilities(x, pi1, pi2, mean1, mean2, covMatrix1, covMatrix2):
    # Calcular p(x_i | mu1, sigma1) e p(x_i | mu2, sigma2)
    p_x_1 = multivariate_normal.pdf(x, mean=mean1, cov=covMatrix1)
    p_x_2 = multivariate_normal.pdf(x, mean=mean2, cov=covMatrix2)
    print(f"P(X|K1) = {p_x_1:.3f} - P(X|K2) = {p_x_2:.3f}")


    # Calcular as responsabilidades r_i1 e r_i2
    numerator1 = pi1 * p_x_1
    numerator2 = pi2 * p_x_2
    print(f"P(X|K1)*P(K1) = {numerator1:.4f} - P(X|K2)*P(K2) = {numerator2:.3f}\n")

    denominator = numerator1 + numerator2
    
    p_1 = numerator1 / denominator
    p_2 = numerator2 / denominator
    
    return p_1, p_2

# M-step: Atualizar parâmetros com base nas responsabilidades

# Função para atualizar os parâmetros
def update_parameters(responsibilities, observations):
    # Número de observações
    n = len(observations)
    
    # Atualizar pesos (pi_k)
    pi1_new = np.mean(responsibilities[:, 0])
    pi2_new = np.mean(responsibilities[:, 1])
    
    # Atualizar médias (mu_k)
    mu1_new = np.sum(responsibilities[:, 0].reshape(-1, 1) * observations, axis=0) / np.sum(responsibilities[:, 0])
    mu2_new = np.sum(responsibilities[:, 1].reshape(-1, 1) * observations, axis=0) / np.sum(responsibilities[:, 1])
    
    # Atualizar matrizes de covariância (sigma_k)
    sigma1_new = np.zeros((2, 2))
    sigma2_new = np.zeros((2, 2))
    
    for i in range(n):
        diff1 = (observations[i] - mu1_new).reshape(-1, 1)
        sigma1_new += responsibilities[i, 0] * (diff1 @ diff1.T)
        
        diff2 = (observations[i] - mu2_new).reshape(-1, 1)
        sigma2_new += responsibilities[i, 1] * (diff2 @ diff2.T)
    
    sigma1_new /= np.sum(responsibilities[:, 0])
    sigma2_new /= np.sum(responsibilities[:, 1])
    
    return pi1_new, pi2_new, mu1_new, mu2_new, sigma1_new, sigma2_new

# Converter lista de observações para array
observations = np.array(observations)

# Iterar duas vezes
for i in range(2):

    print(f"Iteração {i+1}:\n")
    
    # E-step
    responsibilities = []
    for obs in observations:
        r1, r2 = calc_responsibilities(obs, pi1, pi2, mu1, mu2, sigma1, sigma2)
        responsibilities.append([r1, r2])
        print(f"Probabilidade para x_{i}: P_1 = {r1:.3f}, P_2 = {r2:.3f}\n")
    responsibilities = np.array(responsibilities)
    
    # M-step
    pi1, pi2, mu1, mu2, sigma1, sigma2 = update_parameters(responsibilities, observations)
    
    # Imprimir os novos parâmetros
    print(f"Novos pesos: pi1 = {pi1:.3f}, pi2 = {pi2:.3f}")
    print(f"Novas médias: mu1 = {mu1}, mu2 = {mu2}")
    print(f"Novas covariâncias: sigma1 =\n{sigma1}\nsigma2 =\n{sigma2}\n")
    print()




Iteração 1:

P(X|K1) = 0.029 - P(X|K2) = 0.062
P(X|K1)*P(K1) = 0.0147 - P(X|K2)*P(K2) = 0.031

Probabilidade para x_0: P_1 = 0.322, P_2 = 0.678

P(X|K1) = 0.005 - P(X|K2) = 0.048
P(X|K1)*P(K1) = 0.0024 - P(X|K2)*P(K2) = 0.024

Probabilidade para x_0: P_1 = 0.092, P_2 = 0.908

P(X|K1) = 0.036 - P(X|K2) = 0.011
P(X|K1)*P(K1) = 0.0180 - P(X|K2)*P(K2) = 0.005

Probabilidade para x_0: P_1 = 0.770, P_2 = 0.230

Novos pesos: pi1 = 0.394, pi2 = 0.606
Novas médias: mu1 = [ 2.22333752 -0.49554251], mu2 = [0.75368099 0.87317329]
Novas covariâncias: sigma1 =
[[ 1.18237286 -0.84937428]
 [-0.84937428  0.71448514]]
sigma2 =
[[ 0.9467165  -1.03862938]
 [-1.03862938  1.36445026]]


Iteração 2:

P(X|K1) = 0.119 - P(X|K2) = 0.149
P(X|K1)*P(K1) = 0.0469 - P(X|K2)*P(K2) = 0.090

Probabilidade para x_1: P_1 = 0.342, P_2 = 0.658

P(X|K1) = 0.001 - P(X|K2) = 0.209
P(X|K1)*P(K1) = 0.0005 - P(X|K2)*P(K2) = 0.127

Probabilidade para x_1: P_1 = 0.004, P_2 = 0.996

P(X|K1) = 0.346 - P(X|K2) = 0.011
P(X|K1)*P(K1) =

## Exercicio 2

In [2]:
import numpy as np
from scipy.stats import multivariate_normal

# Função para fazer a atribuição "hard" das observações aos clusters com base no MAP
def hard_assignment(observations, pi1, pi2, mu1, mu2, sigma1, sigma2):
    cluster_assignments = []
    for obs in observations:
        # Calcular as probabilidades condicionais para cada cluster
        p_x_given_mu1 = multivariate_normal.pdf(obs, mean=mu1, cov=sigma1)
        p_x_given_mu2 = multivariate_normal.pdf(obs, mean=mu2, cov=sigma2)
        print(f"P(X|K1) = {p_x_given_mu1}, P(X|K2) = {p_x_given_mu2}")
        
        # Multiplicar pelas probabilidades a priori (os pesos pi_k)
        prob1 = pi1 * p_x_given_mu1
        prob2 = pi2 * p_x_given_mu2
        print(f"P(X|K1)P(K1) = {prob1}, P(X|K2)P(K2) = {prob2}\n")

        
        # Atribuir ao cluster com maior probabilidade (MAP)
        if prob1 > prob2:
            cluster_assignments.append(1)
        else:
            cluster_assignments.append(2)
    
    return cluster_assignments

# Atribuir as observações aos clusters
cluster_assignments = hard_assignment(observations, pi1, pi2, mu1, mu2, sigma1, sigma2)
cluster_assignments


P(X|K1) = 0.5641734858046435, P(X|K2) = 0.30449400405811056
P(X|K1)P(K1) = 0.24436147240701495, P(X|K2)P(K2) = 0.1726079706961807

P(X|K1) = 7.939247260676204e-78, P(X|K2) = 0.4726282481421226
P(X|K1)P(K1) = 3.438740386133597e-78, P(X|K2)P(K2) = 0.2679179284920627

P(X|K1) = 1.9032071057089328, P(X|K2) = 1.3452461132198738e-08
P(X|K1)P(K1) = 0.8243395025613951, P(X|K2)P(K2) = 7.625772547084151e-09



[1, 2, 1]